In [ ]:
%matplotlib inline
import logging
import os

import numpy as np
from obspy import UTCDateTime
import pandas as pd
import xarray as xr

from vumt.data import DataSource
from vumt.rsam import rsam
from ipywidgets import FloatProgress
from IPython.display import display

In [ ]:
def save_rsam(df, filename, tr):
    xdf = df.to_xarray()
    xdf = xdf.rename(dict(index='datetime'))
    xdf.attrs['starttime'] = tr.stats.starttime.isoformat()
    xdf.attrs['endtime'] = tr.stats.endtime.isoformat()
    xdf.to_netcdf(filename)

## Test RSAM on a single day

In [ ]:
starttime = UTCDateTime(year=2013, julday=240)
endtime = starttime + 86400.
ds = DataSource(test=True)
stream = 'WIZ.10.HHZ.NZ'
outdir = '/scratch/VUMT/features/RSAM'

site, loc, comp, net = stream.split('.')
for tr in ds.get_waveforms(net, site, loc, comp,
                      starttime, endtime):
    start_dt = tr.stats.starttime.strftime('%Y%m%d%H%M%S')
    end_dt = tr.stats.endtime.strftime('%Y%m%d%H%M%S')
    fout = "{}_{}_{}_rsam.nc".format(start_dt,end_dt,stream.replace('.', '_'))
    fout = os.path.join(outdir, fout)
    df = rsam(tr, filtertype='bp', filterfreq=(2,5))
    save_rsam(df, fout)

In [ ]:
foo = xr.open_dataarray(fout)
foo.to_pandas().plot()

### Test on single days with gaps

In [ ]:
starttime = UTCDateTime(2010, 8, 9, 0, 0, 0)
endtime = starttime + 86400.
stream = 'NZ.WIZ.10.HHZ'
ds1 = DataSource()
net, site, loc, comp = stream.split('.')
for tr1 in ds1.get_waveforms(net, site, loc, comp, starttime, endtime):
    df = rsam(tr1, filtertype='bp', filterfreq=(2,5))
df.plot()

In [ ]:
starttime = UTCDateTime(2010, 11, 26, 0, 0, 0)
endtime = starttime + 86400.
for tr2 in ds1.get_waveforms(net, site, loc, comp, starttime, endtime):
    df = rsam(tr2, filtertype='bp', filterfreq=(2,5), interval=3600.)
df.plot()

## Computer RSAM for 39 days

In [ ]:
ds1 = DataSource(test=True)
starttime = UTCDateTime(year=2013, julday=240)
endtime = UTCDateTime(year=2013, julday=279)
chunk_size = 86400.
rs = []
max_value = (endtime - starttime)/chunk_size
f = FloatProgress(min=0, max=max_value)
display(f)

i = 0
for tr in ds1.get_waveforms('NZ', 'WIZ', '10', 'HHZ', starttime, endtime,
                            chunk_size=chunk_size):
    rs.append(rsam(tr, filtertype='bp', filterfreq=(2,5)))
    i += 1
    f.value = i

In [ ]:
rs_df = pd.concat(rs)
rs_df.plot()

## The whole dataset

In [ ]:
ds2 = DataSource(test=False)
starttime = UTCDateTime(year=2001, julday=1)
endtime = UTCDateTime(year=2019, julday=248)
chunk_size = 86400.
streams = ['WIZ.10.HHZ.NZ', 'WSRZ.10.HHZ.NZ', 'KRVZ.10.EHZ.NZ']
outdir = '/scratch/VUMT/features/RSAM'

for stream in streams[2:]:
    site, loc, comp, net = stream.split('.')
    max_value = (endtime - starttime)/chunk_size
    logging.basicConfig(filename=site+'_rsam.log', level=logging.DEBUG)
    f = FloatProgress(min=0, max=max_value)
    display(f)
    i = 0
    st = starttime
    try:
        for tr in ds2.get_waveforms(net, site, loc, comp, starttime, endtime,
                                    chunk_size=chunk_size):
            try:
                startdate = st.strftime('%Y%m%d')
                fout = "{}_{}.nc".format(startdate, site)
                fout = os.path.join(outdir, site, fout)
                df = rsam(tr, filtertype='bp', filterfreq=(2,5))
                save_rsam(df, fout)
            except Exception as e:
                logging.debug(e)
                logging.debug(starttime)
            st += chunk_size
            i += 1
            f.value = i
    except Exception as e:
        logging.debug(st)
        logging.debug(e)